<a href="https://colab.research.google.com/github/sheikhrakib0/RAGHSC/blob/main/10MRAGPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AI Engineer (Level-1) — Technical Assessment**

**Project Structure**


**Step 1:** Data Loading and Preprocessing Section

**Step 2:** Chunking for retrieval process

**Step 3:** Embedding

**Step 4:** RETRIEVE Function

**Step 5:** Generating Funtion

**Step 6:** RAG Pipeline

**Step 7:** Testing with Sample


**Bonus Task:**
Time shortage (:

In [ ]:
import numpy as np
import pandas as pd

**Step 1: Data Loading and Preprocessing Section**

In [ ]:
#First we need to download the pdf from the location then preprocess it
import gdown

# Replacing with Google Drive file ID
google_drive_file_id = '19h7t3xVNBS5KR9o2sLMh3i2uaje7ww7n'
output_filename = 'downloaded_pdf.pdf'

gdown.download(f'https://drive.google.com/uc?id={google_drive_file_id}', output_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=19h7t3xVNBS5KR9o2sLMh3i2uaje7ww7n
To: /content/downloaded_pdf.pdf
100%|██████████| 1.26M/1.26M [00:00<00:00, 13.5MB/s]


'downloaded_pdf.pdf'

In [ ]:
# For pdf Loading
######################## Finally something working Perfectly ###################################
# The technique used is Optical Character Recognition (OCR) with tesseract, which extracts
# text from the image representation of the PDF rather than relying on embedded text data.
!sudo apt install tesseract-ocr -y
!sudo apt install libtesseract-dev -y

!pip install pymupdf  #for pdf to image before OCR
!sudo apt install tesseract-ocr-ben -y  # For Bengali (ben)
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.5 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB

In [ ]:
import pytesseract

# Set the Tesseract path explicitly (Colab installs it in /usr/bin/tesseract)
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [ ]:
import re

#Cleaning Function
def clean_text_for_rag(text):
    # Normalize multiple line breaks to paragraph breaks
    text = text.replace('\r', '')
    text = re.sub(r'\n{3,}', '\n\n', text)   # 3+ newlines → 2 newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)  # single \n not between paragraphs → space
    return text.strip()

In [ ]:
import fitz  # PyMuPDF
from PIL import Image
import pytesseract

doc = fitz.open("downloaded_pdf.pdf")

In [ ]:

# Open the PDF and extract text
def preprocessor(doc):
  full_text = ""
  for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    pix = page.get_pixmap(dpi=300)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    text = pytesseract.image_to_string(img, lang='ben')
    full_text += text + "\n\n"
    cleaned_text = clean_text_for_rag(full_text)
  return cleaned_text

In [ ]:
#checking
cleaned_text = preprocessor(doc) #doc should be defined first. it's the raw file

In [ ]:
# Save the extracted text for future use
with open("cleaned.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

**Loading and Preprocessing Complete**

The Text file contains clean, formated text

In [ ]:
# Load the text from the cleaned.txt file from the local, if stored
try:
    with open("cleaned.txt", "r", encoding="utf-8") as f:
        cleaned_text = f.read()
    print("Successfully loaded text from ocr_text_pymupdf.txt")
except FileNotFoundError:
    print("Error: ocr_text_pymupdf.txt not found.")
    cleaned_text = None

# Display the first 1000 characters to verify
if cleaned_text:
    print(cleaned_text[:500])

Successfully loaded text from ocr_text_pymupdf.txt
অপরিচিতা

অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,

কল করো 16919

আনলাইন ব্যাচ”

শিখনফল
1. নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে। 
2. তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে। 
3. তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।
4. তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে।
5. নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে পারবে। 
6. মানুষ আশা নিয়ে বেঁচে 


**Step 2: Chunking for retrieval process**

N.B: Chunking has been done by character here. There are more chunking methods.


In [ ]:
# as a text splitter we are using CharacterTextSplitter from Langchain framework
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# chunk by character
def split_docs(cleaned_text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.create_documents([cleaned_text])
    return chunks

In [ ]:
#chunk by sentences
import re

def sentence_chunker(text, max_chunk_size=1000):
    # Step 1: Split text into sentences
    sentences = re.split(r'(?<=[।!?])\s+', text.strip())

    # Step 2: Combine sentences into chunks under the size limit
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # If adding sentence keeps chunk under limit
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    # Append remaining chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


In [ ]:
# testing the function
all_chunks = split_docs(cleaned_text)
all_chunks2 = sentence_chunker(cleaned_text)

In [ ]:
all_chunks[2]

Document(metadata={}, page_content='ফলের মতো হয় তাহলে তার অসম্পূর্ণ সারবস্তা প্রকট হয়ে\n\nওঠে। নিজের নি্ষল জীবনকে বোঝাতে অনুপমের ব্যবহৃত পমা।\n\n \n\nঅন্নপূর্ণা\n\nঅন্নে পরিপূর্ণা। দেবী দুর্গা\n\nদেবী দুর্গার দুই পুত্র; অগ্রজ গণেশ ও অনুজ কাতিকেয়। দুর্গার কোলে থাকা দেব-সেনাপতি কার্তিকেয়কে বোঝানো হয়েছে। ব্যঙ্গার্থে প্রয়োগ।\n\n \n\nভূষণ, প্রসাধন, শোভা। ভাষার মাধুর্য ও উৎকর্ষ বৃদ্ধি করে এমন গুণ।\n\n \n\nভারতের গয়া অঞ্চলের অন্তঃসলিলা নদী। নদীটির ওপরের অংশে বালির আস্তরণ কিন্তু ভিতরে জলম্বোত প্রবাহিত।\n\n \n\nঅনুপম তার মামার চরিত্র-বৈশিষ্ট্য প্রসঙ্গে কথাটি বলেছে। সংসারের সমস্ত দায়-দায়িত্ব পালনে তার ভূমিকা এখানে উপমার মাধ্যমে ব্যক্ত করা হয়েছে\n\n \n\nস্বয়ংবরা\n\nযে মেয়ে নিজেই স্বামী নির্বাচন করে\n\n \n\nগুড়গুড়ি\n\nআলবোলা। ফরসি। দীর্ঘ নলযুক্ত হুকাবিশেষ\n\n \n\nবাঁধা হুকা\n\nসাধারণ মানুষের ব্যবহার্য নারকেল-খোলে তৈরি ধূমপানের যন্ত্রবিশেষ\n\n \n\nউমেদারি\n\nপ্রার্থনা। চাকরির আশায় অন্যের কাছে ধরনা দেওয়া।\n\n \n\nঅবকাশের মরুভূমি এক কালে ইহাদের বংশে লক্ষ্বীর\n\nমঙ্গলঘট ভরা ছিল।')

**Step 3: Embedding**

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
#Another embedding model from Transformers
embed_model = SentenceTransformer("intfloat/multilingual-e5-base")  # Supports Bengali & English


In [ ]:
def embed_chunks(chunks, embed_model= embed_model):
    texts = [chunk.page_content for chunk in chunks]
    embeddings = embed_model.encode(texts, convert_to_tensor=True)
    return embeddings

In [ ]:
#checking
embedded_chunks = embed_chunks(all_chunks)

In [ ]:
len(embedded_chunks)

99

**Step 4: RETRIEVE Function**

Using faiss for fast semantic search.

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.1 MB/s eta 0:00:00


In [ ]:
import faiss

def build_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.cpu().detach().numpy())
    return index


In [ ]:
#checking the function
index = build_index(embedded_chunks)

In [ ]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c96957e5170> >

Building the Retrieval Function

-> Retrieving top 3 chunks by semantic search

In [ ]:
def retrieve_top_k(query, embed_model, chunks, embeddings, index, k=3):
    query_embedding = embed_model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
    distances, indices = index.search(query_embedding, k)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    return retrieved_chunks


In [ ]:
#checking
query = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
retrieved_chunks = retrieve_top_k(query, embed_model, all_chunks, embedded_chunks, index, k=3)

In [ ]:
len(retrieved_chunks)

3

**Step 5: RAG = Generating Answer**

In [ ]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
#Need API Key to use the model
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ··········


In [ ]:
from langchain.chat_models import init_chat_model

In [ ]:
# Final context from top k chunks
def final_context(chunks):
  context = "\n\n".join([chunk.page_content for chunk in chunks])
  return context

In [ ]:
context = final_context(retrieved_chunks)

In [ ]:
context

'32 \x0c\n\n05026. ত্ললাইল ভ্রয়নটা" 1110) 19৫10\n\n৯। অনুপমের আসল অভিভাবক কে?\n\n(ক) বাবা (খ) মামা (গ) মা (ঘ) শিক্ষক\n\n১০। \'অপরিচিতা\' গল্পে মামার সাথে অনুপমের বয়সের পার্থক্য কত?\n\n(ক) বছর চারেক (খ) বছর ছয়েক (গ) বছর আষ্টেক (ঘ) বছর দশেক ১১। কন্যার পিতামাত্রই কোনটি স্বীকার করবেন?\n\n(ক) অনুপম রুচিবান (খ) অনুপম সৎপাত্র\n\n(গ) অনুপম রূপবান (ঘ) অনুপম ব্যক্তিত্বসম্পন্ন\n\n১২। অনুপম কোনটি খায় না বলে গর্ব প্রকাশ করেছে?\n\n(ক) তামাক (খ) মদ (গ) চুরুট (ঘ) কফি\n\n১৩। বিয়েবাড়িতে ঢুকে মামার খুশি না হওয়ার কারণ ছিল না কোনটি?\n\n(ক) স্থান ও আয়োজন দেখে (খ) আপ্যায়নের ত্রুটির কারণে\n\n(গ) গহনার পরিমাণ দেখে (ঘ) বেয়াইয়ের আচর-আচরণে\n\n১৪। মামা কেমন ঘরের মেয়ে পছন্দ করতেন?\n\n(ক) ধনী (খ) গরিব (গ) গ্রামীণ (ঘ) শহুরে\n\n১৫। অনুপমের বন্ধুর নাম কী?\n\n(ক) সতীশ (খ) জ্যোতিষ (গ) হরিশ (ঘ) মণীষ\n\n১৬। মেয়ের চেয়ে মেয়ের বাপের খবরটাই কার কাছে গুরুতর?\n\n(ক) হরিশের (খ) অনুপমের (গ) মামার (ঘ) ঘটকের ১৭। অনুপমের শিক্ষাগত যোগ্যতা কি?\n\n(ক) বিএ পাশ (খ) এমএ পাশ (গ) বিএসসি পাশ (ঘ) এমএসসি পাশ ১৮। \'মেয়ে যদি বলো, 

In [ ]:
from langchain_core.prompts import PromptTemplate
def prompt_template(query, context):
  prompt_temp = PromptTemplate(
    template = """
    You are a helpful assistant.
    Answer only from the provided context.
    context: {context}
    Question: {question}
    """,
    input_variables = ["context", "question"]
  )
  prompt =  prompt_temp.invoke({"context": context, "question": query})
  return prompt

In [ ]:
#checking
prompt = prompt_template(query, context)
prompt

StringPromptValue(text='\n    You are a helpful assistant.\n    Answer only from the provided context.\n    context: (ক) গণেশ (খ) প্রজাপতি (গ) কার্তিক (ঘ) পঞ্চশর \n\n৩৯। \'এখানে জায়গা আছে\' উক্তিটি কার?\n\n(ক) আর্দালির (খ) গার্ডের (গ) কল্যাণীর ৪০। স্টেশনে অনুপম কী ফেলে গেল? (ক) টিকিট (খ) ক্যামেরা (গ) তোরঙ্গ\n\n৪১। ট্রেনে দেখা হওয়ার সময় কল্যাণীর বয়স কত ছিল? (ক) ১৪/১৫ বছর (খ) ১৫/১৬ বছর (গ) ১৬/১৭ বছর ৪২। অপরিচিতা মেয়েটির সঙ্গে কতজন মেয়ে ছিল?\n\n(ক) ২/৩ জন (খ) ৩/৪ জন (গ) ৪/৫ জন ৪৩। কল্যাণী স্টেশন হতে কী খাবার কিনে নেয়?\n\n(ক) চানা-মুঠ (খ) ঝালমুড়ি (গ) চিনেবাদাম ৪৪। শস্তুনাথ পেশায় কী ছিলেন?\n\n(ক) উকিল (খ) শিক্ষক (গ) ডাক্তার\n\n৪৫। মাতৃ-আজ্ঞা বলতে কল্যাণী কার প্রতি ইঙ্গিত করেছে?\n\n(ক) মায়ের প্রতি (খ) মাতৃভূমির প্রতি (গ) ধরণীর প্রতি ৪৬। বিবাহের সময় অনুপমের বয়স কত ছিল?\n\n(ক) ২১ বছর (খ) ২৩ বছর (গ) ২৫ বছর ৪৭। গজাননের মায়ের নাম কী?\n\n(ক) অন্নদা (খ) অন্নপূর্ণা (গ) কল্যাণী \n\n৪৮। \'শিগগির চলে আয়, এই গাড়িতে জায়গা আছে\' উক্তিটি কার? (ক) অনুপমের (খ) কল্যাণীর (গ) বিনুদাদার ৪৯। হরিশ 

In [ ]:
model = init_chat_model("gemini-2.0-flash-001", model_provider="google_genai")

In [ ]:
model.invoke(prompt)

AIMessage(content='১৬ বছর', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--f2dc572d-dd7a-4601-9d85-791f5886b570-0', usage_metadata={'input_tokens': 1820, 'output_tokens': 6, 'total_tokens': 1826, 'input_token_details': {'cache_read': 0}})

There are other models. Gemini gives us free api, we are using the Gemini 2.0 flash

In [ ]:
# generate answer funtion

def generate_answer(query, retrieved_chunks):
    contexts = final_context(retrieved_chunks)
    prompt = prompt_template(query, contexts)
    model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
    result = model.invoke(prompt)
    return result.content

In [ ]:
#checking
result = generate_answer(query, retrieved_chunks)

**Step 6: RAG Pipeline**

In [ ]:
## RAG Pipeline
def rag_pipeline(query, doc_file):
    #cleaned_text = preprocessor(doc_file)
    chunks = split_docs(doc_file)
    embed_model = SentenceTransformer("intfloat/multilingual-e5-base")  # Supports Bengali & English
    embeddings = embed_chunks(chunks, embed_model)
    index = build_index(embeddings)
    retrieved_chunks = retrieve_top_k(query, embed_model, chunks, embeddings, index)
    answer = generate_answer(query, retrieved_chunks)
    return answer

In [ ]:
import fitz
query1 = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
doc = fitz.open("/content/downloaded_pdf.pdf")
with open("cleaned.txt", 'r', encoding='utf-8') as f:
        text_file = f.read()


In [ ]:
answer1 = rag_pipeline(query1, text_file)
answer1

'context এ এই প্রশ্নের উত্তর নেই।'

In [ ]:
## Question 2
query2 = "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?"
doc = fitz.open("/content/downloaded_pdf.pdf")
with open("cleaned.txt", 'r', encoding='utf-8') as f:
        text_file = f.read()

In [ ]:
answer2 = rag_pipeline(query2, text_file)
answer2

'মামাকে'

In [ ]:
## Question 3
query3 = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
doc = fitz.open("/content/downloaded_pdf.pdf")
with open("cleaned.txt", 'r', encoding='utf-8') as f:
        text_file = f.read()

In [ ]:
answer3 = rag_pipeline(query3, text_file)
answer3

In [ ]:
## Question 1 English
query1_en = "Who is called a good man in Anupam's language?"
doc = fitz.open("/content/downloaded_pdf.pdf")
with open("cleaned.txt", 'r', encoding='utf-8') as f:
        text_file = f.read()

In [ ]:
answer1_En = rag_pipeline(query1_en, text_file)
answer1_En

In [ ]:
## Question 2 English
query2_en = "Who is referred to as the god of luck of Anupam?"
doc = fitz.open("/content/downloaded_pdf.pdf")
with open("cleaned.txt", 'r', encoding='utf-8') as f:
        text_file = f.read()

In [ ]:
answer2_En = rag_pipeline(query2_en, text_file)
answer2_En

In [ ]:
## Question 3 English
query3_en = "What was Kalyani's actual age at the time of marriage?"
doc = fitz.open("/content/downloaded_pdf.pdf")
with open("cleaned.txt", 'r', encoding='utf-8') as f:
        text_file = f.read()

In [ ]:
answer3_En = rag_pipeline(query3_en, text_file)
answer3_En

'Answer: (গ) ১৬ বছর'

**This is the wastege section where I tried different methods to complete each steps, some of them worked poorly and some of them didn't worked out. So I keep them in this section if I need anything later from here, I can use.**

**Data loading and preprocessing Wastage**

In [ ]:

#!pip install PyPDF2
#!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
from PyPDF2 import PdfReader

In [ ]:
def extract_text_from_pdfs(file_path):

    if file_path.endswith('.pdf') and os.path.exists(file_path):
        reader = PdfReader(file_path)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    else:
        print(f"Error: File not found or not a PDF at {file_path}")
    return text

In [ ]:
all_text = extract_text_from_pdfs(output_filename)

In [ ]:
# Trying another method
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.5 MB/s eta 0:00:00


In [ ]:
from pdfminer.high_level import extract_text

def extract_text_with_pdfminer(file_path):
    if os.path.exists(file_path) and file_path.endswith('.pdf'):
        try:
            text = extract_text(file_path)
            return text
        except Exception as e:
            print(f"Error extracting text with pdfminer.six: {e}")
            return None
    else:
        print(f"Error: File not found or not a PDF at {file_path}")
        return None

# Use the function with your downloaded file
pdfminer_text = extract_text_with_pdfminer(output_filename)

if pdfminer_text:
    print("Text extracted using pdfminer.six:")
    print(pdfminer_text[:1000]) # Print first 1000 characters to check
else:
    print("Failed to extract text using pdfminer.six")

Text extracted using pdfminer.six:
িাাং া
১ম পত্র

আল ািয রিষয়

অপরিরিতা

অনলাইন ব্যাচ সম্পর্কি ত যেককাকনা জিজ্ঞাাসা ,

র্িখনফল

✓ র্নম্নর্ব্ি ব্যজক্তি হঠাৎ র্ব্িিালী হক  ওঠাি ফকল সমাকি পর্িচ  সংকট সম্পককি ধািণা লাভ কিকব্।
✓ তৎকালীন সমাি-সভযতা ও মানব্তাি অব্মাননা সম্পককি িানকত পািকব্।
✓ তৎকালীন সমাকিি পণপ্রথাি কু প্রভাব্ সম্পককি িানকত পািকব্।
✓ তৎকাকল সমাকি ভদ্রকলাককি স্বভাব্বব্র্িষ্ট্য সম্পককি জ্ঞাানলাভ কিকব্।
✓ নািী যকামল ঠিক, র্কন্তু দুব্িল ন - কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত 

পািকব্।
✓ মানুষ আিা

র্নক  যব্ঁকচ থাকক- অনুপকমি দৃষ্ট্াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি

জ্ঞাানলাভ কিকব্।

প্রাক-মূলযা ন

১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?

ক) ডাক্তার্ি

খ) ওকালর্ত

গ) মাস্টার্ি

ঘ) ব্যব্সা

২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-

ক) প্রর্তপজি

খ) প্রভাব্

গ) র্ব্চক্ষণতা

ঘ) কূ ট ব্ুর্ি

র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।

র্পতৃ হীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতি া। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র

In [ ]:
 # another method
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.5 MB/s eta 0:00:00


In [ ]:
import pdfplumber

# Path to your PDF file
pdf_path = "downloaded_pdf.pdf"

# Open and extract text
full_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:  # Check if text was extracted
            full_text += text + "\n"

# Save to a text file (optional)
with open("extracted_text.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

# Print the extracted text (optional)
print(full_text[0:2000])


িাাং া
১ম পত্র
আল ািয রিষয়
অপরিরিতা
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,
র্িখনফল
✓ র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ সংকটসম্পককি ধািণালাভকিকব্।
✓ তৎকালীনসমাি-সভযতাওমানব্তািঅব্মাননাসম্পককি িানকতপািকব্।
✓ তৎকালীনসমাকিিপণপ্রথািকুপ্রভাব্সম্পককি িানকতপািকব্।
✓ তৎকাকলসমাকিভদ্রকলাককিস্বভাব্বব্র্িষ্ট্যসম্পককি জ্ঞাানলাভকিকব্।
✓ নািী যকামল ঠিক, র্কন্তু দব্ু লি ন - কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত
পািকব্।
✓ মানুষ আিা র্নক যব্কঁ চ থাকক- অনপু কমি দষ্ট্ৃ াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি
জ্ঞাানলাভকিকব্।
প্রাক-মূলযা ন
১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?
ক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা
২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-
ক) প্রর্তপজি খ) প্রভাব্ গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি
র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।
র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা
তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অ

In [ ]:
# Another method
!pip install pytesseract pdf2image pillow


In [ ]:
import pytesseract

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'


In [ ]:
import os
import sys

# Ensure Windows recognizes .exe executables
if sys.platform.startswith("win"):
    os.environ["PATH"] += os.pathsep + r"C:\Users\Sheikh Rakib\Downloads\poppler\bin"


In [ ]:
from pdf2image import convert_from_path
import pytesseract

# For Windows:
poppler_path = r"C:/Users/Sheikh Rakib/Downloads/poppler/bin"

# Convert to images using Poppler
pages = convert_from_path('downloaded_pdf.pdf', dpi=300, poppler_path=poppler_path)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [ ]:
from pdf2image import convert_from_path
import pytesseract
import os

# Optional: Set path to tesseract executable on Windows
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Convert PDF to images
pages = convert_from_path('downloaded_pdf.pdf', dpi=300)

# OCR with Bengali language
full_text = ""
for i, page in enumerate(pages):
    text = pytesseract.image_to_string(page, lang='ben')
    full_text += text + "\n\n"

# Save to a .txt file
with open("ocr_extracted_bengali.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

print("✅ Bengali OCR extraction complete.")
#didn't work out

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

**Embedding Wastage**

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

There are several vector stores like Chroma, FAISS, Lance.